# Reading and Writing the Apache Parquet Format

## Existing open standards
- XML, json
- SQL
- binary storage format with metadata (NetCDF, HDF5, Apache Parquet)
- serialization/ RPC protocols (Apache AVRO, protocol buffers)


## Why we need open standards?
Performance, no overhead, valid accross programming language

## Which benefits for Pandas?
Not based originally on open standards, that's it!

## Why columnar tables?
- SQL is row oriented format (ex. Apache Impala, PostgreSQL)
- but queries are often made on columns of a table, or on a subset of the columns.

## The Apache Arrow project
- **Goal**: Define an open standard for column-oriented tables (data frames) that is language-independant (Java, Python, R, Javascript, ...), so **portable accross languages**

- need specifications, libraies, tools

## Resources

* Apache Parquet format: https://github.com/apache/parquet-format
* videos: 
  * 2018-07 by Wes McKinney: https://www.youtube.com/watch?v=y7zGnKzaKIw (existing standards, challenges)
  * 2019-06 by Wes McKinney: https://www.youtube.com/watch?v=uZA55cGDaBQ

## Reading and Writing Single Files

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa

In [ ]:

df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                  index=list('abc'))

df

In [ ]:
table = pa.Table.from_pandas(df)

In [ ]:
table

In [ ]:
# We write this to Parquet format with write_table:
import pyarrow.parquet as pq
pq.write_table(table, 'example.parquet')

In [ ]:
# This creates a single Parquet file.
# In practice, a Parquet dataset may consist of many files
# in many directories.
!ls

In [ ]:
# We can read a single file back with read_table:
table2 = pq.read_table('example.parquet')

In [ ]:
table2.to_pandas()

In [ ]:
# You can pass a subset of columns to read,
# which can be much faster than reading the whole file
# (due to the columnar layout):
pq.read_table('example.parquet', columns=['one', 'three']).to_pandas()

In [ ]:
# When reading a subset of columns from a file that used
# a Pandas dataframe as the source,
# we use read_pandas to maintain any additional index column data:
pq.read_pandas('example.parquet', columns=['two']).to_pandas()

## Omitting the DataFrame index

In [ ]:
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                  index=list('abc'))

df

In [ ]:
table = pa.Table.from_pandas(df, preserve_index=False)

In [ ]:
pq.write_table(table, 'example_noindex.parquet')
t = pq.read_table('example_noindex.parquet')
t.to_pandas()
# Here you see the index did not survive the round trip